In [1]:
import numpy as np 
import pandas as pd 
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('../input/covid19-global-forecasting-week-2/train.csv')
test_data = pd.read_csv('../input/covid19-global-forecasting-week-2/test.csv')

In [3]:
features = train_data 
features = features.drop(columns=['Province_State','ConfirmedCases','Fatalities'])

In [4]:
features.Date = pd.to_datetime(features.Date)
features.Date = features.Date.astype(int)
features.head()

,Id,Country_Region,Date
0,1,Afghanistan,1579651200000000000
1,2,Afghanistan,1579737600000000000
2,3,Afghanistan,1579824000000000000
3,4,Afghanistan,1579910400000000000
4,5,Afghanistan,1579996800000000000


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
features.Country_Region = le.fit_transform(features.Country_Region)
features.info()
features.head(200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20580 entries, 0 to 20579
Data columns (total 3 columns):
Id                20580 non-null int64
Country_Region    20580 non-null int64
Date              20580 non-null int64
dtypes: int64(3)
memory usage: 482.5 KB


,Id,Country_Region,Date
0,1,0,1579651200000000000
1,2,0,1579737600000000000
2,3,0,1579824000000000000
3,4,0,1579910400000000000
4,5,0,1579996800000000000
...,...,...,...
195,256,2,1584403200000000000
196,257,2,1584489600000000000
197,258,2,1584576000000000000
198,259,2,1584662400000000000


In [6]:
target_con = train_data.ConfirmedCases
target_con.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: ConfirmedCases, dtype: float64

In [7]:
test_features = test_data.drop(columns=['Province_State'])
test_features.Date = pd.to_datetime(test_features.Date)
test_features.Date = test_features.Date.astype(int)
test_features.Country_Region = le.fit_transform(test_features.Country_Region)
test_features.info()
test_features.head(200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12642 entries, 0 to 12641
Data columns (total 3 columns):
ForecastId        12642 non-null int64
Country_Region    12642 non-null int64
Date              12642 non-null int64
dtypes: int64(3)
memory usage: 296.4 KB


,ForecastId,Country_Region,Date
0,1,0,1584576000000000000
1,2,0,1584662400000000000
2,3,0,1584748800000000000
3,4,0,1584835200000000000
4,5,0,1584921600000000000
...,...,...,...
195,196,4,1586563200000000000
196,197,4,1586649600000000000
197,198,4,1586736000000000000
198,199,4,1586822400000000000


In [8]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,random_state=10)
rf.fit(features,target_con)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [9]:
predict = rf.predict(test_features)

predict

array([22., 24., 24., ...,  1.,  1.,  1.])

In [10]:
target_fat = train_data.Fatalities
target_fat.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: Fatalities, dtype: float64

In [11]:
rf.fit(features,target_fat)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [12]:
predict_fat = rf.predict(test_features)

predict_fat

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
submit = pd.read_csv('../input/covid19-global-forecasting-week-2/submission.csv')
submit.ForecastId = test_data.ForecastId
submit.ConfirmedCases = predict
submit.Fatalities = predict

submit.head(25)

,ForecastId,ConfirmedCases,Fatalities
0,1,22.0,22.0
1,2,24.0,24.0
2,3,24.0,24.0
3,4,40.0,40.0
4,5,40.0,40.0
5,6,74.0,74.0
6,7,84.0,84.0
7,8,94.0,94.0
8,9,110.0,110.0
9,10,110.0,110.0


In [14]:
submit.to_csv('submission.csv',index=False)